# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`course_details_merged.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../data/toorandmed_aasta.csv`

**Väljund:** `../data/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [44]:
import pandas as pd
import json
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = 'data/toorandmed_aasta.csv'
OUTPUT_FILE = 'data/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile
df.head(2)

print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

Toorandmed loetud. Andmestiku suurus: 3416 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


### Samm 1: eemaldame andmed, mida me ei taha

Siin võib filtreerida välja ridu mõne tunnuse alusel kui teame, et oma rakenduses vajame ainult teatud tüüpi andmeid, hetkel me seda ei teinud, aga kui tahaksime nt ainult kevade aineid, saaksime seda siin teha.

In [45]:
# Filtreeime välja ained, mis on kaitsmisega seotud või kompleksained ja mille kestus on 1 semester
target_types = ['regular', 'practice']
df = df[(df['additional_info__duration_in_semesters'] == 1) & (df['general__type__code'].isin(target_types))]
print(f"Filtreerimine tehtud. Järelejäänud ridu: {len(df)}")

Filtreerimine tehtud. Järelejäänud ridu: 2900


### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [46]:
def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """

    # 1. Veerud, kus ühendame üldise info ja versiooni info (eelistades versiooni)
    # (uus nimi, üldise info veerg, versiooni info veerg)
    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('eap', 'credits', 'version__credits'),
        ('kirjeldus', 'overview__description__et', 'version__overview__description__et'),
        ('opivaljundid', 'overview__learning_outcomes_text_et', 'version__overview__learning_outcomes_text_et')
    ]

    # 2. Veerud, mis võetakse ühest konkreetsest kohast (lihtne ümbernimetamine)
    # (uus nimi, vana nimi)
    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries'),
        ('keel', 'version__target__language__et'),
        ('hindamisskaala', 'additional_info__assessment_scale__et'),
        ('asukoht', 'version__target__course_main_structural_unit__city'),
        ('oppeaste_json', 'version__additional_info__study_levels'),
        ('hindmaismeetod', 'version__grading__grade_evaluation__et'),
        ('miinimumnouded', 'version__grading__grade_preconditions__et')
    ]

    print("Alustan veergude filtreerimist ja ühendamist...")
    
    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False
        
        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan
    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan
    return df


# Rakendame funktsiooni
df_resolved = resolve_fields(df.copy())

# Kontrollime tulemust
print(f"Samm 2: Puhastatud veergude lisamine tehtud.")
print(df_resolved[['aine_kood', 'semester']].isnull().sum())
print(df_resolved['semester'].unique())

Alustan veergude filtreerimist ja ühendamist...
Samm 2: Puhastatud veergude lisamine tehtud.
aine_kood     0
semester     14
dtype: int64
['kevad' 'sügis' nan]



### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [47]:
# --- Abifunktsioonid JSON töötlemiseks ---

def parse_json_safe(json_str):
    """
    Teisendab JSON-stringi turvaliselt Pythoni objektiks (list või dict).
    Käsitleb tühje väärtusi (NaN, None) ja katkist JSON-it, tagastades vea korral None.
    """
    if pd.isna(json_str) or json_str == '': 
        return None
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None


def extract_lecturers(json_str):
    """
    Eraldab JSON-struktuurist õppejõudude nimed ('person_name').
    Tagastab komadega eraldatud stringi unikaalsetest nimedest tähestikulises järjekorras.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    names = [p.get('person_name') for p in data if isinstance(p, dict) and p.get('person_name')]
    return ", ".join(sorted(list(set(names)))) if names else None
 

def extract_schedule_days_et(json_str):
    """
    Analüüsib tunniplaani aegu ja leiab, mis nädalapäevadel aine toimub.
    1. Parsib kuupäeva/kellaaja.
    2. Teisendab päeva nime eesti keelde (nt Monday -> Esmaspäev).
    3. Sorteerib päevad loogilises järjekorras (Esmaspäevast Pühapäevani), mitte tähestiku järgi.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    
    days = set()
    day_map = {
        'Monday': 'Esmaspäev', 'Tuesday': 'Teisipäev', 'Wednesday': 'Kolmapäev',
        'Thursday': 'Neljapäev', 'Friday': 'Reede', 'Saturday': 'Laupäev', 'Sunday': 'Pühapäev'
    }
    
    for entry in data:
        if not isinstance(entry, dict): continue
        time_str = entry.get('time') or entry.get('start_time')
        if time_str:
            dt = pd.to_datetime(time_str, errors='coerce')
            if not pd.isna(dt):
                en_day = dt.day_name()
                days.add(day_map.get(en_day, en_day))
    
    week_order = ['Esmaspäev', 'Teisipäev', 'Kolmapäev', 'Neljapäev', 'Reede', 'Laupäev', 'Pühapäev']
    sorted_days = sorted(list(days), key=lambda d: week_order.index(d) if d in week_order else 99)
    
    return ", ".join(sorted_days) if sorted_days else None

def extract_study_levels_et(json_str):
    """
    Eraldab JSON-struktuurist õppeastmed.
    Tagastab komadega eraldatud stringi unikaalsetest õppeastmetest.
    """
    data = parse_json_safe(json_str)
    if not data: return None

    values = [item.get('et') for item in data if isinstance(item, dict) and item.get('et')]
    return ", ".join(sorted(list(set(values)))) if values else None

def extract_prerequisites(json_str):
    """
    Võtab JSON-ist eeldusainete koodid ja nimed ning paneb need kokku stringiks.
    Näide väljundist: "LTAT.01.001 Programmeerimine, MTAT.03.100 Andmebaasid"
    """
    data = parse_json_safe(json_str)
    if not data: return None
    
    try:
        # Puhastame stringi, kui seal on topelt-jutumärke (faili eripära)
        cleaned_str = json_str.replace('""', '"')
        data = json.loads(cleaned_str)
        
        res = []
        for item in data:
            code = item.get('code', '')
            name = item.get('title', {}).get('et', '')
            
            if code and name:
                res.append(f"{code} {name}")
            elif code:
                res.append(code)
                
        return ", ".join(res) if res else None
    except:
        return None

# --- Funktsioonide rakendamine ---
print("Ekstraheerime JSON väljadest infot...")

df_resolved['oppejoud'] = df_resolved['oppejoud_json'].apply(extract_lecturers)
df_resolved['toimumisajad'] = df_resolved['toimumisajad_json'].apply(extract_schedule_days_et)
df_resolved['oppeaste'] = df_resolved['oppeaste_json'].apply(extract_study_levels_et)
df_resolved['eeldusained'] = df_resolved['additional_info__prerequisites'].apply(extract_prerequisites)
  
print("Samm 3: JSON töötlemine valmis.")
print(df_resolved[['aine_kood', 'oppejoud', 'toimumisajad', 'oppeaste', 'eeldusained']].head(3))


Ekstraheerime JSON väljadest infot...
Samm 3: JSON töötlemine valmis.
     aine_kood  \
0  OIEO.06.046   
2  KKSB.05.092   
3  ARKI.02.030   

                                                                             oppejoud  \
0                                                                           Gea Lepik   
2                                                                    Kadri Medijainen   
3  Jekaterina Nerman, Jürgen Rünk, Pille-Riin Värk, Ragnar Lõivukene, Viljo Kübarsepp   

         toimumisajad                                    oppeaste  \
0                None                                 magistriõpe   
2  Neljapäev, Laupäev                             bakalaureuseõpe   
3                None  integreeritud bakalaureuse- ja magistriõpe   

                      eeldusained  
0                            None  
2                            None  
3  AR00.00.016 Diagnostika alused  


### Samm 4 (Valikuline): Info koondamine mitmest allikast

Kui vajalik info on mitmes veerus, saame need ühendada. Hetkel teeme seda näiteks hindamisinfoga.

### Samm 5: Lõplik viimistlus

Valime lõplikud veerud. 

In [48]:
final_cols = [
    'aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester',
    'kirjeldus', 'hindamine_info','toimumisajad','oppejoud', 
    'opivaljundid', 'keel', 'hindamisskaala', 'asukoht', 
    'oppeaste', 'hindmaismeetod', 'miinimumnouded', 'eeldusained'
]


existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print("Samm 5: Lõplik andmestik koostatud.")

Lõplik ridade arv: 2900, lõplik veergude arv: 15
Samm 5: Lõplik andmestik koostatud.


### Samm 6: Salvestamine ja kontroll

Kontrollime mõningaid kvaliteediaspekte ning salvestame puhastatud faili.

* Salvesta puhastatud fail.
* Arvuta puuduvate väärtuste hulk igas veerus.
* Leia kategooriliste tunnustega veergude [keel, semester, õppeaste, …] enim levinud väärtused (näiteks df[tunnus].count().head(5))
* Näita "Sissejuhatus andmeteadusesse” aine kogu puhastatud kujul väljund lihtsasti loetaval kujul (võid küsida LLMilt abi, et väljund hästi kenasti vormistada, et see ei oleks lihtsalt välja prinditud andmerida).
* Lisada tunnus kirjeldus, sisaldab ühe aine koguinfot (kõik veerud ühendatud suureks tekstiks. Teha tunnuse kirjeldus tähemärkide arvu statistika (describe() funktsioon võib aidata, aga võib kasutada ka visualiseerimist), et teaksime, kui palju infot me RAGile ette anda kavatseme

In [49]:
# salvesta fail
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")

#TODO

Samm 6: Andmestik salvestatud faili 'data/andmed_aasta.csv'.


### Samm 7: Info puhastatud andmete kohta

In [50]:
print("--- Puuduvate väärtuste hulk ja osakaal veergudes ---")
missing_data = pd.DataFrame({
    'Puuduvaid väärtusi': df_final.isnull().sum(),
    'Osakaal (%)': (df_final.isnull().sum() / len(df_final) * 100).round(2)
})

# Näitame ainult neid veerge, kus on vähemalt üks puuduv väärtus
print(missing_data[missing_data['Puuduvaid väärtusi'] > 0].sort_values(by='Puuduvaid väärtusi', ascending=False))

categorical_cols = ['keel', 'semester', 'oppeaste', 'toimumisajad', 'hindamisskaala', 'asukoht'] 

print("\n--- Kategooriliste tunnuste TOP 5 väärtust ---")
for col in categorical_cols:
    if col in df_final.columns:
        print(f"\nVeerg: {col}")
        print(df_final[col].value_counts().head(5))

from IPython.display import display, Markdown

# Leiame aine (eeldame, et aine_kood on veeru nimi)
aine_row = df_final[df_final['aine_kood'] == 'LTAT.02.002']

if not aine_row.empty:
    aine = aine_row.iloc[0]
    
    # Koostame ilusa esituse
    html_output = f"## 📘 Aine detailvaade: {aine['nimi_et']} ({aine['aine_kood']})\n\n"
    html_output += "| Väli | Sisu |\n| :--- | :--- |\n"
    
    for col in final_cols:
        if col in aine:
            väärtus = aine[col] if pd.notna(aine[col]) else "*Puudub*"
            html_output += f"| **{col}** | {väärtus} |\n"
    
    display(Markdown(html_output))
else:
    print("\nAinet 'LTAT.02.002' ei leitud.")

def koonda_tekst(row):
    tekstiosad = []
    for col in final_cols:
        if col in row and pd.notna(row[col]):
            tekstiosad.append(f"{col}: {row[col]}")
    return "\n".join(tekstiosad)

df_final['kirjeldus_koond'] = df_final.apply(koonda_tekst, axis=1)

df_final['kirjelduse_pikkus'] = df_final['kirjeldus_koond'].str.len()

print("\n--- Tunnuse 'kirjeldus_koond' tähemärkide statistika ---")
print(df_final['kirjelduse_pikkus'].describe())

--- Puuduvate väärtuste hulk ja osakaal veergudes ---
                Puuduvaid väärtusi  Osakaal (%)
eeldusained                   2370        81.72
toimumisajad                  1836        63.31
opivaljundid                  1560        53.79
asukoht                       1073        37.00
miinimumnouded                 361        12.45
hindmaismeetod                 347        11.97
oppeaste                        47         1.62
oppejoud                        15         0.52
semester                        14         0.48
keel                            14         0.48

--- Kategooriliste tunnuste TOP 5 väärtust ---

Veerg: keel
keel
eesti keel      2132
inglise keel     754
Name: count, dtype: int64

Veerg: semester
semester
kevad    1524
sügis    1362
Name: count, dtype: int64

Veerg: oppeaste
oppeaste
bakalaureuseõpe                               905
magistriõpe                                   535
bakalaureuseõpe, magistriõpe                  306
integreeritud bakalaureuse- 